# Creación dataset

In [8]:
import json
from faker import Faker
import random
import sys

# Inicializar Faker
fake = Faker()

num_inserciones = 2000000

# Definir posiciones y equipos aleatorios
posiciones = ['Delantero', 'Centrocampista', 'Defensa', 'Portero']
equipos = ['Equipo A', 'Equipo B', 'Equipo C', 'Equipo D']

# Crear una lista para almacenar los datos generados
datos = []

# Generar datos aleatorios
for _ in range(num_inserciones):
    jugador = {
        "nombre": fake.name(),
        "posicion": random.choice(posiciones),
        "equipo": random.choice(equipos),
        "fecha_nacimiento": fake.date_of_birth(minimum_age=18, maximum_age=40).isoformat()
    }
    datos.append(jugador)

# Guardar los datos en un archivo JSON
with open('jugadores.json', 'w') as file:
    json.dump(datos, file, indent=4)

print(f"Datos generados y guardados en jugadores.json")


Datos generados y guardados en jugadores.json


# Cargar dataset Mysql

In [9]:
import mysql.connector
from mysql.connector import Error
import json
import sys

# Cargar los datos generados desde el archivo JSON
with open('jugadores.json', 'r') as file:
    datos = json.load(file)

# Conectar a la base de datos MySQL e insertar datos
try:
    connection = mysql.connector.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        password='root',
        database='ejercicio'
    )

    if connection.is_connected():
        print("Conexión exitosa a la base de datos MySQL")

        # Crear un cursor para ejecutar consultas SQL
        cursor = connection.cursor()

        # Insertar datos en la tabla jugadores
        for jugador in datos:
            query = """
            INSERT INTO jugadores (nombre, posicion, equipo, fecha_nacimiento)
            VALUES (%s, %s, %s, %s);
            """
            values = (jugador['nombre'], jugador['posicion'], jugador['equipo'], jugador['fecha_nacimiento'])
            cursor.execute(query, values)

        # Confirmar la inserción de los datos
        connection.commit()
        print(f"Datos insertados correctamente en la tabla jugadores")

except Error as e:
    print("Error al conectar a MySQL:", e)

finally:
    # Cerrar la conexión
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Conexión MySQL cerrada")



Conexión exitosa a la base de datos MySQL
Datos insertados correctamente en la tabla jugadores
Conexión MySQL cerrada


# Cargar dataset Cassandra

In [10]:
import json
import os
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from uuid import uuid4
from datetime import datetime

# Configuración de conexión con Cassandra
def conectar_cassandra():
    # Aquí se asume que tienes acceso a Cassandra de manera local o en la red
    cluster = Cluster(['localhost'])  # Cambia esto si tu Cassandra está en otro servidor
    session = cluster.connect()
    session.set_keyspace('ejercicio')
    return session

# Función para insertar un jugador en la base de datos
def insertar_jugador(session, jugador):
    # Insertar los datos en la tabla 'jugadores'
    query = """
    INSERT INTO jugadores (id, nombre, posicion, equipo, fecha_nacimiento)
    VALUES (%s, %s, %s, %s, %s)
    """
    # Convertir la fecha de nacimiento de string a un objeto de fecha
    fecha_nacimiento = datetime.strptime(jugador['fecha_nacimiento'], '%Y-%m-%d').date()
    
    # Ejecutar la consulta con los valores
    session.execute(query, (uuid4(), jugador['nombre'], jugador['posicion'], jugador['equipo'], fecha_nacimiento))

# Función para leer el archivo JSON y procesar los jugadores
def procesar_json(json_data):
    session = conectar_cassandra()

    for jugador in json_data:
        insertar_jugador(session, jugador)

    print("Datos insertados correctamente.")

if __name__ == "__main__":
    json_path = ('jugadores.json')  # Ruta al archivo

    # Cargar el archivo JSON con la información de los jugadores
    with open(json_path, 'r') as f:
        jugadores_json = json.load(f)

    # Procesar e insertar los jugadores
    procesar_json(jugadores_json)


Datos insertados correctamente.


# Consulta de datos en Mysql

In [10]:
import mysql.connector
from mysql.connector import Error
import time

try:
    # Establece la conexión con la base de datos
    connection = mysql.connector.connect(
        host='127.0.0.1',     # O usa 'localhost'
        port=3306,            # Puerto expuesto por Docker
        user='root',          # Usuario de MySQL (cambia si usaste otro)
        password='root',      # Contraseña del usuario (ajusta a la tuya)
        database='ejercicio'  # Nombre de la base de datos
    )
    
    if connection.is_connected():
        print("Conexión exitosa a la base de datos MySQL")

        # Crear un cursor para ejecutar consultas SQL
        cursor = connection.cursor()
        
        # Medir el tiempo de ejecución de la consulta
        start_time = time.time()  # Iniciar el temporizador
        
        # Ejemplo de consulta: Selecciona todos los jugadores
        cursor.execute("SELECT * FROM jugadores;")
        rows = cursor.fetchall()

        # Calcular el tiempo de ejecución
        elapsed_time = time.time() - start_time
        print(f"Tiempo de ejecución de la consulta: {elapsed_time:.4f} segundos")

        print("Datos en la tabla jugadores:")
        for i in range(5):
            print(rows[i])
            
        

except Error as e:
    print("Error al conectar a MySQL:", e)

finally:
    # Cierra la conexión
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Conexión MySQL cerrada")


Conexión exitosa a la base de datos MySQL
Tiempo de ejecución de la consulta: 8.0027 segundos
Datos en la tabla jugadores:
(1, 'Samantha Foster', 'Defensa', 'Equipo A', datetime.date(1987, 8, 14))
(2, 'Lance Beard', 'Defensa', 'Equipo D', datetime.date(1997, 12, 15))
(3, 'Robert Mcclain', 'Defensa', 'Equipo B', datetime.date(1998, 9, 16))
(4, 'Elizabeth Brooks', 'Portero', 'Equipo C', datetime.date(1991, 2, 26))
(5, 'Michael Washington', 'Defensa', 'Equipo C', datetime.date(2001, 11, 9))
Conexión MySQL cerrada


# Consulta de datos en Cassandra

In [ ]:
import time
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from uuid import UUID

# Función para conectar a Cassandra
def conectar_cassandra():
    cluster = Cluster(['localhost'])  # Cambia esto si tu Cassandra está en otro servidor
    session = cluster.connect()
    session.set_keyspace('ejercicio')
    return session

# Función para consultar todos los jugadores
def consultar_jugadores(session):
    start_time = time.time()  # Iniciar el temporizador
    
    # Consulta para obtener todos los jugadores
    query = "SELECT * FROM jugadores ALLOW FILTERING"
    rows = session.execute(query)

    # Calcular el tiempo de ejecución
    elapsed_time = time.time() - start_time
    print(f"Tiempo de ejecución de la consulta (todos los jugadores): {elapsed_time:.4f} segundos")

    # Imprimir los resultados
    for i, row in enumerate(rows):
        if i <= 5:  # Obtén las primeras 5 filas
            print(f"ID: {row.id}, Nombre: {row.nombre}, Posición: {row.posicion}, Equipo: {row.equipo}, Fecha Nacimiento: {row.fecha_nacimiento}")
        else:
            break

# Función para consultar un jugador específico por nombre
def consultar_jugador_por_nombre(session, nombre):
    start_time = time.time()  # Iniciar el temporizador
    
    query = "SELECT * FROM jugadores WHERE nombre = %s ALLOW FILTERING"
    rows = session.execute(query, (nombre,))

    # Calcular el tiempo de ejecución
    elapsed_time = time.time() - start_time
    print(f"Tiempo de ejecución de la consulta (jugador {nombre}): {elapsed_time:.4f} segundos")

    for row in rows:
        print(f"ID: {row.id}, Nombre: {row.nombre}, Posición: {row.posicion}, Equipo: {row.equipo}, Fecha Nacimiento: {row.fecha_nacimiento}")

# Función principal
if __name__ == "__main__":
    # Conectar a Cassandra
    session = conectar_cassandra()

    # Consultar todos los jugadores
    print("Consultando todos los jugadores:")
    consultar_jugadores(session)



Consultando todos los jugadores:
Tiempo de ejecución de la consulta (todos los jugadores): 0.0731 segundos
ID: e76aea66-dd8a-4927-a6aa-dbe25bf72b1b, Nombre: Brenda Guzman, Posición: Centrocampista, Equipo: Equipo C, Fecha Nacimiento: 1999-06-28
ID: 82f3bb27-4237-45ec-ae84-d9721cfddb23, Nombre: Tina Allen, Posición: Centrocampista, Equipo: Equipo B, Fecha Nacimiento: 1999-09-16
ID: bffdb637-c00a-4302-89be-659c41a13bf5, Nombre: Zachary Shelton, Posición: Portero, Equipo: Equipo C, Fecha Nacimiento: 2006-08-09
ID: 56d9ff75-20fe-460f-be5a-d61420356d8c, Nombre: Christine Hernandez, Posición: Defensa, Equipo: Equipo B, Fecha Nacimiento: 1983-11-25
ID: 39637ffd-3143-4b42-8808-61ac97863ccf, Nombre: Joanne Moore, Posición: Portero, Equipo: Equipo C, Fecha Nacimiento: 1989-06-24
ID: 65945c30-3e49-4b8b-a04e-2c871a7d9d01, Nombre: Alyssa Hogan, Posición: Portero, Equipo: Equipo B, Fecha Nacimiento: 1995-08-12


# Conclusión

Cassandra parece más rápido para hacer consultas. El resultado esperado era que MySql sería más rápido ya que la consulta realizada fue para obtener todas las filas y Cassandra en lo que se especializa es en las columnas.

Investigando un poco parece que los motivos son:

Optimización para consultas masivas: Cassandra está diseñada para escalar y realizar operaciones rápidas de lectura incluso con grandes volúmenes de datos.

Lectura eficiente desde memoria y almacenamiento secuencial: Cassandra maneja los datos en memoria (memtable) y en disco (SSTables), lo que hace que las lecturas sean mucho más rápidas.

Menor overhead: Cassandra tiene menos overhead en comparación con MySQL, que realiza más comprobaciones y operaciones internas.

Escalabilidad: Aunque estás usando un solo nodo en Cassandra, la base de datos está diseñada para operar eficientemente incluso en un solo nodo.